# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [8]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [9]:
data = pd.read_csv('customer_product_sales.csv')

In [11]:
data.head(20)

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20
5,83106,Hope,Bowers,6484922,229,Bread - Hot Dog Buns,22
6,11253,Nicolas,Holmes,4878478,229,Bread - Hot Dog Buns,3
7,35107,Judith,Miranda,2353899,229,Bread - Hot Dog Buns,9
8,15088,Ruby,Lowe,37021,229,Bread - Hot Dog Buns,4
9,26031,Josephine,Curry,2327232,229,Bread - Hot Dog Buns,7


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [33]:
mydataframe = data.groupby(["CustomerID", "ProductName"]).agg({'Quantity':'sum'})
mydataframe.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [38]:
filtered = data[(data["CustomerID"] == 97753) & (data["ProductName"] == "Anchovy Paste - 56 G Tube")]
filtered

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
3910,97753,Randi,Suarez,1668362,131,Anchovy Paste - 56 G Tube,25


In [36]:
mydataframe2 = pd.pivot_table(mydataframe, values = "Quantity", index = "ProductName", columns = "CustomerID", 
                              fill_value = 0)
mydataframe2.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [47]:
from scipy.spatial.distance import pdist, squareform

sim_matrix = squareform(pdist(mydataframe2, 'euclidean'))
sim_matrix

array([[  0.        , 279.99642855, 253.66907577, ..., 255.67166444,
        258.0910692 , 278.47800631],
       [279.99642855,   0.        , 271.49953959, ..., 265.7367118 ,
        279.55500353, 286.80655502],
       [253.66907577, 271.49953959,   0.        , ..., 245.27535547,
        253.01580978, 269.43644891],
       ...,
       [255.67166444, 265.7367118 , 245.27535547, ...,   0.        ,
        264.14200726, 268.28343221],
       [258.0910692 , 279.55500353, 253.01580978, ..., 264.14200726,
          0.        , 273.15014186],
       [278.47800631, 286.80655502, 269.43644891, ..., 268.28343221,
        273.15014186,   0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [53]:
distances = pd.DataFrame(sim_matrix)

display(distances)

,0,1,2,3,4,5,6,7,8,9,...,442,443,444,445,446,447,448,449,450,451
0,0.000000,279.996429,253.669076,238.512054,242.810626,275.550358,264.474952,257.029181,247.519696,247.570192,...,262.608073,265.744238,258.980694,246.704276,259.474469,247.382295,254.911749,255.671664,258.091069,278.478006
1,279.996429,0.000000,271.499540,269.455006,269.838841,292.006849,286.438475,255.432966,262.922042,285.854858,...,273.822935,286.251638,274.952723,265.625676,273.830239,284.024647,269.896276,265.736712,279.555004,286.806555
2,253.669076,271.499540,0.000000,235.210544,258.810742,262.891613,260.862032,241.722982,242.317973,270.272085,...,254.108245,270.510628,261.306334,225.341962,239.681038,242.235423,243.191283,245.275355,253.015810,269.436449
3,238.512054,269.455006,235.210544,0.000000,244.235542,250.659131,261.600841,238.872351,249.110417,259.613174,...,253.978346,258.530462,251.414001,218.753286,243.530286,257.297493,243.991803,247.737765,256.894920,267.985074
4,242.810626,269.838841,258.810742,244.235542,0.000000,265.343174,267.675176,244.497444,255.759653,263.711964,...,264.979244,271.556624,257.487864,235.244554,259.703677,267.968282,247.864883,251.855117,261.400077,264.497637
5,275.550358,292.006849,262.891613,250.659131,265.343174,0.000000,266.411336,255.980468,272.921234,270.294284,...,269.000000,272.594938,282.759615,257.761518,258.969110,267.305069,266.514540,257.243076,266.523920,279.699839
6,264.474952,286.438475,260.862032,261.600841,267.675176,266.411336,0.000000,258.412461,250.605267,282.223316,...,270.806204,271.707563,273.642102,261.009578,262.571895,267.804033,253.911402,263.068432,270.181421,285.220967
7,257.029181,255.432966,241.722982,238.872351,244.497444,255.980468,258.412461,0.000000,244.090147,264.081427,...,260.597391,254.625215,258.520792,230.436542,240.222813,246.519776,244.413584,246.734675,248.501509,261.224042
8,247.519696,262.922042,242.317973,249.110417,255.759653,272.921234,250.605267,244.090147,0.000000,264.954713,...,257.003891,272.646291,263.076035,234.829726,248.251888,249.843951,251.809452,245.833277,258.222772,265.593675
9,247.570192,285.854858,270.272085,259.613174,263.711964,270.294284,282.223316,264.081427,264.954713,0.000000,...,276.936816,281.110299,275.923903,260.099981,264.018939,263.797271,262.562374,269.653481,274.830857,278.310977


In [49]:
similarities = distances[33].sort_values(ascending=False)[1:]
similarities[:5]

340    279.814224
358    279.381460
217    278.531865
315    278.486984
17     278.037767
Name: 33, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.